In [ ]:
! pip show pandas

# Libraries

In [1]:
import pandas as pd
import numpy  as np
import string 
import re
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
# import nltk
# from nltk.corpus import wordnet
# from nltk.stem import WordNetLemmatizer
# #from nltk.tokenize.treebank import TreebankWordDetokenizer
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
#from nltk.corpus import stopwords

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import gensim.downloader as api

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.text import text_to_word_sequence
# from tensorflow.keras.preprocessing.text import Tokenizer

# from tensorflow.keras import Sequential, layers
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras import models

from sklearn.utils import resample
print('Done')


2022-01-13 21:49:32.348580: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 21:49:32.348603: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Done


In [2]:
from tensorflow.keras import models

# Help functions

## Undersampling

In [3]:
def undersample(df, prop = 2):
    
    # get label 2
    df2 = df[df["label"]==2]
    n2 = df2.shape[0]
    
    # get label 0
    df0= df[df["label"]==0]
    df0 = df0.sample(frac=1)
    
    # sample from label 0
    n0 = round(prop*n2)
    df0_sample = df0.head(n0)
    
    # concatenate
    df = pd.concat([df0_sample,df2], axis = 0)
    df = df.sample(frac=1)
    df.reset_index(inplace = True)
    
    assert(n0==prop*n2)
    return df[["text","score_punctuation", "score_capital_word", "label"]]

## Oversampling

In [4]:
def oversample(df, prop = 0.3):# , seed = 123):
    n = df.shape[0]
    n2_new = round(prop*n) 
    
    # get label 2
    df2 = df[df["label"]==2]
    n2 = df2.shape[0]
    n2_resample = n2_new - n2
    if (n2_resample) < 0:
        print(f"WARNING: proportion {prop} already satisfied, DF unchanged")
    
    else : 
        indices = list(range(n2))
    
        # resample from df2
        indices_resample = resample(indices, replace=True, n_samples =n2_resample)#, random_state=f"{seed}")
        df2_resample = df2.iloc[indices_resample]

        df2_new = pd.concat([df2_resample, df2], axis =0)

        df_new = pd.concat([df2_new, df[df["label"]==0]], axis =0)
        df = df_new
    #assert(df2_new.shape[0]==prop*n)
    df = df.sample(frac=1)
    df.reset_index(inplace = True)
    return df[["text","score_punctuation", "score_capital_word", "label"]]


## Data cleaning and tokenizing function

In [5]:
# # clean each tweet
# def clean_data(data, remove_special_char_2lower_case = True):
    
#     #Removing URLs with a regular expression
#     url_pattern = re.compile(r'https?://\S+|www\.\S+')
#     data = url_pattern.sub(r'', data)

#     # Remove Emails
#     data = re.sub('\S*@\S*\s?', '', data)
   
#     # tokenize + remove scpecial characters + set to lower case
#     if remove_special_char_2lower_case == True:
#         data = text_to_word_sequence(data) 
#     else :
#     # tokenize 
#         data = data.split() 
    
    
#     # Remove stopwords
#     stop_words = set(stopwords.words('english')) 
#     data = [w for w in data if not w in stop_words]         
    
#     # Remove digits
#     data = ' '.join(word for word in data if not word.isdigit())
    
    
#     return text_to_word_sequence(data)

# # clean dataset
# def apply_data_cleaning(X, text, drop_text = False, remove_special_char_2lower_case = True):
#     ln = X.shape[0]
#     sentences = []
#     for i in range(ln):
#         tmp = X.iloc[i][f'{text}']
#         tmp_clean = clean_data(tmp,remove_special_char_2lower_case = f"{remove_special_char_2lower_case}")
#         sentences.append(tmp_clean)
#     X["sentences"] = sentences
#     if drop_text == True:
#         X.drop(columns = f'{text}', inplace = True)
#     return X

In [6]:
def clean_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)
   
    # tokenize + remove scpecial characters + set to lower case
    data = text_to_word_sequence(data) 
    
    
    # Remove stopwords
    stop_words = set(stopwords.words('english')) 
    data = [w for w in data if not w in stop_words]         
    
    # Remove digits
    data = ' '.join(word for word in data if not word.isdigit())
    
    
    return text_to_word_sequence(data)

def apply_data_cleaning(X, text, drop_text = False):
    ln = X.shape[0]
    sentences = []
    for i in range(ln):
       # print (f"{i}")
        tmp = X.iloc[i][f'{text}']
        tmp_clean = clean_data(tmp)
        sentences.append(tmp_clean)
    X["sentences"] = sentences
    if drop_text == True:
        X.drop(columns = f'{text}', inplace = True)
    return X

## Help function for Embedding

In [7]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed



# Help function adding features to embedding

In [8]:
def append_features_to_tensor(X_text, X_feature, _max ):

    tmp = np.empty(shape=(X_text.shape[0],X_text.shape[1]+2, X_text.shape[2] ))
    tmp[:,0:X_text.shape[1],:] = X_text

    first_indices = X_text.shape[0] ## # of twitts 
    tmp_punct =  np.zeros(shape = (1,X_text.shape[2]))
    tmp_cap = np.zeros(shape = (1,X_text.shape[2]))

    for i in range(first_indices):
        if X_feature.iloc[i,0] == 1:
            tmp_punct =  tmp_punct*(_max+0.5)

        if X_feature.iloc[i,1] == 1:
            tmp_cap = tmp_cap*(_max+1)   

        tmp[i,200,:] = tmp_punct
        tmp[i,201,:] = tmp_cap

    
    return tmp

# Data

## Load re-labeled data

In [9]:
with open('../raw_data/DF', 'rb') as handle:
    DF = pickle.load(handle)
DF.dropna(inplace = True)

## Undersample

In [10]:
df = undersample(df= DF, prop = 9)
df.shape
df["label"].value_counts()

0    90819
2    10091
Name: label, dtype: int64

## Oversample

In [11]:
df = oversample(df =df , prop = 0.3 )#, seed = 123)
df.shape
df["label"].value_counts()

0    90819
2    30273
Name: label, dtype: int64

## SubSampling 

In [14]:
N =10000
df = df.head(N)
df.label.value_counts()

0    7517
2    2483
Name: label, dtype: int64

## Replace emojis with text

In [ ]:
import emoji


emoji_replace = []
for tweet in df.text:
    emoji_replace.append((emoji.demojize(tweet, delimiters=(" ", " "))).replace("_",""))

df.text = emoji_replace

In [16]:
print(df.shape)
df.head()
#df.to_csv("../raw_data/df120k_emoji2text")
# #fonction pr récupérer le nouveau df 

# path = '' #add the path of the directory where filter_df.pickle is stored 
# with open(path + 'df2','rb') as read_file:
#     df2_noemoji = pickle.load(read_file)

(10000, 4)


,text,score_punctuation,score_capital_word,label
0,NOT ME. Personal Experience!,0,0,0
1,BUILD THAT DAMN WALL! WHAT YOU WAITING FOR? P...,0,1,2
2,I don’t even know if you will read this but I ...,0,1,2
3,EARLY VOTING Is ON...GO VOTE\n\nWe've 1 Month ...,0,1,2
4,That’s always been wild to me. I was able to u...,0,1,2


# Data split train, test

In [17]:
# Split in X, y
X = df.copy() 
X.drop(columns = "label", inplace = True)
y = df["label"]

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)
assert(len(X_train)==len(y_train))
assert(len(X_test)==len(y_test))
assert(len(X_train)+len(X_test)==X.shape[0])

In [18]:
print(len(y))
print(len(y_train))
print(len(y_test))

10000
8000
2000


In [19]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(10000, 3)
(8000, 3)
(2000, 3)


In [20]:
# Split X into text and features
X_train_text = X_train[["text"]]
X_test_text = X_test[["text"]]

X_train_features = X_train[["score_punctuation", "score_capital_word"]]
X_test_features = X_test[["score_punctuation", "score_capital_word"]]

In [21]:
print(len(y_train))
print(len(y_test))

print(X_train_text.shape)
print(X_train_features.shape)

print(X_test_text.shape)
print(X_test_features.shape)

8000
2000
(8000, 1)
(8000, 2)
(2000, 1)
(2000, 2)


In [22]:
X_train.head()

,text,score_punctuation,score_capital_word
9254,Perhaps they’ll thank when it happens.,0,0
1561,Why people are misusing hastag when it's been...,0,0
1670,"Well, I don't know the details (and neither ...",0,1
6087,[new] MeToo,0,0
6669,Editors Guild salutes women journalists who ha...,0,0


In [23]:
X_test.head()

,text,score_punctuation,score_capital_word
6252,"The estimates are in:\n: 45th year, 210,000 at...",0,0
4684,I thought Matt took his entire family and move...,0,1
1731,Reading stories from victims are utterly dis...,0,0
4742,\n \n\n \n \n \n \n\n \n\n's confirms limit...,0,0
4521,"crossedswords shield \n\nGOD THE FATHER, T...",0,1


# Clean and tokenize text

In [24]:
# print(X_text.head())
X_train_text = apply_data_cleaning(X = X_train_text, text = "text",drop_text = True)#, remove_special_char_2lower_case = True)
X_test_text = apply_data_cleaning(X = X_test_text, text = "text", drop_text = True)#, remove_special_char_2lower_case = True)

X_train_text = list(X_train_text.sentences)
y_train = np.array(y_train)

X_test_text = list(X_test_text.sentences)
y_test = np.array(y_test)

assert(len(X_train_text)==len(y_train))
assert(len(X_test_text)==len(y_test))
assert(X_train_features.shape[0]==len(y_train))
assert(X_test_features.shape[0])==len(y_test)

In [25]:
# print(len(y_train))
# print(len(y_test))

# print(len(X_train_text))
# print(X_train_features.shape)

# print(len(X_test_text))
# print(X_test_features.shape)

# Save cleaned and tokenized data

In [28]:
# # X_train
# with open('../raw_data/X_train_text', 'wb') as f1:
#     pickle.dump(X_train_text, f1)
# with open('../raw_data/X_train_features', 'wb') as f1:
#     pickle.dump(X_train_features, f1)

# # X_test
# with open('../raw_data/X_test_text', 'wb') as f1:
#     pickle.dump(X_test_text, f1)   
# with open('../raw_data/X_test_features', 'wb') as f1:
#     pickle.dump(X_test_features, f1)       

# # y_train
# with open('../raw_data/y_train', 'wb') as f1:
#     pickle.dump(y_train, f1)
    
# # y_test2
# with open('../raw_data/y_test', 'wb') as f1:
#     pickle.dump(y_test, f1)   

In [29]:
print(len(y_train))
print(len(X_train_text))
print(X_train_features.shape)

# print(len(y_test))
# print(len(X_test_text))
# print(X_test_features.shape)

8000
8000
(8000, 2)


# Embedding TF




In [31]:
#print(list(api.info()['models'].keys()))
ll = ['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300', 
 'word2vec-ruscorpora-300', 
 'word2vec-google-news-300', 
 'glove-wiki-gigaword-50', 
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200', 
 'glove-wiki-gigaword-300', 
 'glove-twitter-25', 
 'glove-twitter-50', 
 'glove-twitter-100', 
 'glove-twitter-200', 
 '__testing_word2vec-matrix-synopsis']



In [ ]:
#print(list(api.info()['models'].keys()))
word2vec_transfrer = api.load('glove-twitter-50')

In [33]:
# Embed the training and test sentences
X_train_embed_tf = embedding(word2vec_transfer, X_train_text)
X_test_embed_tf = embedding(word2vec_transfer, X_test_text)

# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed_tf, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed_tf, dtype='float32', padding='post', maxlen=200)

NameError: name 'word2vec_transfer' is not defined

## Pickle and save embedded data 

In [ ]:
print(len(y_train))
print(len(y_test))

print(X_train.shape)
print(X_test.shape)

In [ ]:
# # X_train
# with open('../raw_data/X_train_pad', 'wb') as f1:
#     pickle.dump(X_train_pad, f1)
    
# # X_test
# with open('../raw_data/X_test_pad', 'wb') as f1:
#     pickle.dump(X_test_pad, f1)   
    

# # y_train
# with open('../raw_data/y_train', 'wb') as f1:
#     pickle.dump(y_train, f1)
    
# # y_test2
# with open('../raw_data/y_test', 'wb') as f1:
#     pickle.dump(y_test, f1)   

# Add features to tensor

In [ ]:
X_train_pad_added_features = append_features_to_tensor(X_train_pad, \
                                                       X_feature = X_train_features, \
                                                       _max = X_train_pad.max())

In [ ]:
X_test_pad_added_features = append_features_to_tensor(X_test_pad, \
                                                       X_feature = X_test_features, \
                                                       _max = X_test_pad.max())

# Pickle data

# Run model

In [ ]:
def init_model():
    model = Sequential()
    model.add(layers.Masking())#, label ="layer1_masking")
    model.add(layers.LSTM(20, activation='tanh'))#, label ="layer2_LSTM")
    model.add(layers.Dense(15, activation='relu'))#, label ="layer3_dense_relu")
    model.add(layers.Dense(1, activation='sigmoid'))#, label ="layer4_dense_sigmoid")

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=["Recall"]) #['recall']) # tp/(tp+fn) metrics=[tf.keras.metrics.Recall()]
    
    return model

model = init_model()


In [ ]:
es = EarlyStopping(patience=10, restore_best_weights=True)
#%%time
history = model.fit(X_train_pad, y_train, 
          batch_size = 32,  # low --> optimizes on smaller ss --> faster --> but to generalize 
          epochs=10,
          validation_split=0.3,
          callbacks=[es]
         )


In [ ]:
def plot_loss_recall(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['recall'])
    ax[1].plot(history.history['val_recall'])
    ax[1].set_title('Model Recall')
    ax[1].set_ylabel('recall')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)

In [ ]:
plot_loss_recall(model.history, title=None)

## Select epoch and train

In [ ]:
# Save the model
# filepath = './saved_model'
# save_model(model, filepath)
# model = load_model(filepath, compile = True)

In [ ]:
model = init_model()
model.fit(X_train_pad, y_train, 
          batch_size = 32,  # low --> optimizes on smaller ss --> faster --> but to generalize 
          epochs=10,
          validation_split=0.3#,
        #  callbacks=[es]
         )

# Save model

In [ ]:
! ls ../project_BAN/models

In [ ]:
models.save_model(model, '../project_BAN/models/model')

# Load model

In [ ]:
loaded_model = models.load_model('../project_BAN/models/model')

# Predict from tweet - test

### Prediction time

In [ ]:
# write tweet
tweet = "I love deep learning"

score_punctuation = ponctuation_check(tweet)
score_capitalization = capital_word_check(tweet)

tweet_tokenized = clean_data(tweet)
tweet_tokenized

tweet_emebedded = embed_sentence_with_TF(word2vec_transfer, tweet_tokenized)
tweet_emebedded_reshape  = tweet_emebedded.reshape((1,tweet_emebedded.shape[0], tweet_emebedded.shape[1]))

res = loaded_model.predict(tweet_reshape)
res

In [ ]:
if res[0][0]< 0.5:
    print("good tweet")
else:
    print("hate tweet")

In [ ]:
import gensim.downloader as api
import pickle
word2vec_transfimporer = api.load('glove-twitter-50')
with open('../project_BAN/models/word2vec_transfer', 'wb') as f1:
    pickle.dump(word2vec_transfer, f1, protocol = 4)

In [ ]:
with open('../project_BAN/models/word2vec_transfer', 'wb') as f1:
    pickle.dump(word2vec_transfer, f1, protocol = 4)

In [ ]:
# with open('../project_BAN/models/word2vec_transfer', 'rb') as handle:
   #     word2vec_transfer = pickle.load(handle)

# To do

- Replace emojis by text in traning data

- Prep data upto embedding and pickle  

------ new jn ----------------
- import pickled data + add features
- save as pickle

------------ new jn ------------------
run models 



